# Шикунов Николай ИАД-2

In [2]:
import kaggle

In [0]:
kaggle.api.competition_download_files('recsys-iad-challenge')

In [4]:
! kaggle competitions download -c recsys-iad-challenge

 95% 351M/368M [00:02<00:00, 144MB/s]
100% 368M/368M [00:02<00:00, 153MB/s]
 82% 9.00M/10.9M [00:00<00:00, 92.2MB/s]
100% 10.9M/10.9M [00:00<00:00, 101MB/s] 
 95% 223M/235M [00:01<00:00, 153MB/s]
100% 235M/235M [00:01<00:00, 135MB/s]


In [6]:
! unzip items.json.zip
! unzip recsys-iad-challenge.zip
! unzip train.json.zip

Archive:  items.json.zip
  inflating: items.json              
Archive:  recsys-iad-challenge.zip
  inflating: items/items.json        
  inflating: random_benchmark.csv/random_benchmark.csv  
  inflating: train/train.json        
Archive:  train.json.zip
  inflating: train.json              


In [0]:
! rm -r random_benchmark.csv

In [8]:
! unzip random_benchmark.csv.zip

Archive:  random_benchmark.csv.zip
  inflating: random_benchmark.csv    


In [9]:
!ls

items		random_benchmark.csv	  sample_data  train.json.zip
items.json	random_benchmark.csv.zip  train
items.json.zip	recsys-iad-challenge.zip  train.json


In [0]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm_notebook
import gc
from scipy.sparse import dok_matrix, hstack, identity
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy import sparse

## Собираю матрицу юзер-айтем

In [0]:
user_amount = 0

In [13]:
with open('train.json', 'r') as file:
  for line in tqdm_notebook(file):
    user = json.loads(line)
    user_amount += 1

In [0]:
items_id = set()

In [14]:
with open('items.json', 'r') as file:
  for line in tqdm_notebook(file):
    item = json.loads(line)
    items_id.add(item['itemId'])

In [0]:
X = dok_matrix((user_amount, len(items_id)), dtype=np.float32)
key_error = 0

In [17]:
with open('train.json', 'r') as file:
  for line in tqdm_notebook(file, total=user_amount):
    user = json.loads(line)
    user_id = user['userId']
    for item_id, action in user['trainRatings'].items():
      X[user_id, int(item_id)] = 1 if action else -1

# Айтемы - фичи

In [0]:
items = [json.loads(line) for line in open('items.json', 'r')]
df_items = pd.DataFrame(items, columns=['itemId', 'title', 'content', 'image'])
del items
gc.collect();

In [0]:
df_items = df_items.loc[df_items.itemId.isin(items_id), :]
df_items.index = df_items.itemId
df_items.image = [el if type(el) == list else [0]*96 for el in df_items.image]

In [0]:
# В итоге не юзал картинки, очень долго иначе обучается лайтфм
image_imbed = sparse.csr_matrix(np.array(df_items.image.tolist()), dtype=np.float32)

In [0]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), max_df=0.8, norm='l2', use_idf=True, sublinear_tf=True, max_features=15000, analyzer='word')
transformed_content = tfidf.fit_transform(df_items.content)
tfidf = TfidfVectorizer(ngram_range=(1, 2), max_df=0.8, norm='l2', use_idf=True, sublinear_tf=True, max_features=15000, analyzer='word')
transformed_title = tfidf.fit_transform(df_items.title)

In [0]:
item_features = hstack([identity(df_items.shape[0]), transformed_content, transformed_title])

In [23]:
item_features.shape

(328050, 358050)

In [25]:
del df_items, transformed_title, transformed_content, image_imbed
gc.collect()

0

In [0]:
gc.collect();

# Модель - lightfm

In [27]:
! pip install lightfm

     |████████████████████████████████| 307kB 3.4MB/s 
  Created wheel for lightfm: filename=lightfm-1.15-cp36-cp36m-linux_x86_64.whl size=707602 sha256=f2fbb78f239708de995419b403da3f1803b72b5ba584169bbd28c18610137f1f
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [0]:
from lightfm import LightFM

В самом лучшем скоре - 70 компонент, лосс логистик и 12 эпох

In [0]:
model = LightFM(no_components=90, loss='logistic')

In [0]:
for _ in tqdm_notebook(range(2)): 
  model.fit_partial(X,
          item_features=item_features,
          epochs=5, num_threads=4)

In [0]:
random_prediction = pd.read_csv('random_benchmark.csv')

random_prediction.loc[:, 'score'] = model.predict(random_prediction.userId.values, random_prediction.itemId.values,
                                                                 item_features=item_features, 
                                                                 num_threads=4)

In [0]:
prediction = random_prediction.groupby('userId').apply(lambda x: x.sort_values(['score'], ascending=False)).reset_index(drop=True)
prediction.drop('score', axis=1).to_csv("submission.csv", index=False)

In [0]:
! kaggle competitions submit -c recsys-iad-challenge -m 'Shikunov Nikolay IAD2' -f submission.csv